In [ ]:
import os
import pathlib
import json

import pandas as pd

In [ ]:
ru_or_eng = 1

BASE_DIR = str(pathlib.Path().resolve())
TARGET_DIR = ''
WORKING_DIR = f'{BASE_DIR}/{TARGET_DIR}'

dir_list = os.listdir(WORKING_DIR)

mapping = ''
mapping_dict = {}

i = 1

for file in dir_list:
    if file.endswith(".xlsx"):
        mapping_dict[i] = file
        i += 1

for key in mapping_dict.keys():
    print(key,': ', mapping_dict[key])

#
# print(f'BASE_DIR: {BASE_DIR}')
# print(f'TARGET_DIR: {TARGET_DIR}')
# print(f'WORKING_DIR: {WORKING_DIR}')
# print(f'dir_list: {dir_list}')

mapping_key = input('\nWrite mapping number:\n')

mapping = mapping_dict[int(mapping_key)]

map_extr = mapping.split('.')

print(map_extr[0])

print(f'mapping: {mapping}')

In [ ]:
df = pd.read_excel(f'{WORKING_DIR}/{mapping}', sheet_name='Mapping',
                   usecols="D,T:V,Z,AA")

df = df.drop(0,axis=0)

df.columns = ['SchemaS', 'SchemaT', 'Table', 'Code', 'Data Type', 'Length']

df = df[df['Code']!='hdp_processed_dttm']

df = df.sort_values(['Table'])

df.index = range(1, len(df) + 1)

df.head()

In [ ]:
tables_lst = df['Table'].unique()
print(
    len(tables_lst)
)
print(tables_lst)

In [ ]:
test_df = df[df['Table']=='accMark_1kv17'].copy()
test_df = test_df.fillna('')
test_df

In [ ]:
test_row = test_df.iloc[0]
schema_s = test_row['SchemaS']
print(schema_s)
schema_t = test_row['SchemaT']
print(schema_t)
table = test_row['Table']
print(table)
columns = []
columns_casts = []

for _, row in test_df.iterrows():
    columns.append(row['Code'])
    columns_casts.append(
        {
            "name": row['Code'],
            "colType": row['Data Type']
        }
    )

print(columns)
print(columns_casts)

In [ ]:
"""
select
cast([Year] as int ) as 'Year', 
cast([Month] as int ) as 'Month', 
cast([Kvartal] as nvarchar(20) ) as 'Kvartal', 
cast([Date] as date ) as 'Date', 
cast([ACCNT_20] as nvarchar(255) ) as 'ACCNT_20', 
cast([ACCNT_GID] as nvarchar(255) ) as 'ACCNT_GID', 
cast([Comment] as nvarchar(255) ) as 'Comment', 
cast([VGO_OMO] as nvarchar(255) ) as 'VGO_OMO', 
cast([Segment_IFRS] as nvarchar(255) ) as 'Segment_IFRS', 
cast([Division_flag] as nvarchar(10) ) as 'Division_flag', 
cast([Segment_final] as nvarchar(255) ) as 'Segment_final', 
cast([Business_final] as nvarchar(255) ) as 'Business_final', 
cast([Analytical_Srez] as nvarchar(255) ) as 'Analytical_Srez', 
cast([Business2019] as nvarchar(50) ) as 'Business2019' 
from $schema.$table
"""

In [ ]:
test_row = test_df.iloc[0]
schema_s = test_row['SchemaS']
print(schema_s)
schema_t = test_row['SchemaT']
print(schema_t)
table = test_row['Table']
print(table)

query_full = ''
query_prefix = 'select '
query_suffix = ' from $schema.$table'
query_cast_list = []

for _, row in test_df.iterrows():
    attr = row['Code']
    typ = row['Data Type']
    length = f"({test_row['Length']})" if test_row['Length'] else ''
    query_cast_list.append(f"cast([{attr}] as {typ}{length} ) as '{attr}'")

query_full = ', '.join(query_cast_list)

query_full = query_prefix + query_full + query_suffix

print(query_full)

In [ ]:
test_flow_entity_lst = []

template1 = {
    "loadType": "Scd1Replace",
    "source": {
        "schema": schema_s,
        "table": table,
        "columns": columns,
        "columnCasts": columns_casts,
        "jdbcDialect": "..."
    },
    "target": {
        "table": table
    }
}

test_flow_entity_lst.append(template1)

main_json_template = {
  "connection": {
    "connType": "jdbc",
    "url": "...",
    "driver": "...",
    "user": "...",
    "password": "..."
  },
  "commonInfo": {
    "targetSchema": schema_t,
    "etlSchema": schema_t,
    "logsTable": "logs597"
  },
  "flows": test_flow_entity_lst
}

json.dumps(main_json_template)

In [ ]:
def print_flow_entity(schema_s, table, columns, columns_casts):
	template = {
		"loadType": "Scd1Replace",
		"source": {
			"schema": schema_s,
			"table": table,
			"columns": columns,
			"columnCasts": columns_casts,
			"jdbcDialect": "..."
		},
		"target": {
			"table": table
		}
	}

	return json.dumps(template)

json_str = print_flow_entity(schema_s, table, columns, columns_casts)

json_str